In [ ]:
user_regions = ["Asia", "Europe", "America"]
data_centers = {
    "DC-Asia": {"region": "Asia"},
    "DC-Europe": {"region": "Europe"},
    "DC-America": {"region": "America"}
}


In [ ]:
def route_request(user_region):
    # Basic 1-to-1 mapping
    return {
        "Asia": "DC-Asia",
        "Europe": "DC-Europe",
        "America": "DC-America"
    }[user_region]


In [ ]:
import random
import time
import threading
from queue import Queue

class DataCenter:
    def __init__(self, name):
        self.name = name
        self.queue = Queue()
        self.total_response_time = 0
        self.total_requests = 0

    def process_requests(self):
        while True:
            req = self.queue.get()
            response_time = random.uniform(0.2, 1.0)  # Simulate time
            time.sleep(response_time)
            self.total_response_time += response_time
            self.total_requests += 1
            print(f"{self.name} served {req} in {response_time:.2f} sec", flush=True)



In [ ]:
def user_simulator(region, data_centers):
    for i in range(10):
        request_id = f"{region}-REQ{i}"
        dc_name = route_request(region)
        data_centers[dc_name].queue.put(request_id)
        time.sleep(random.uniform(0.1, 0.3))


In [ ]:
# Create DC objects
dc_objects = {name: DataCenter(name) for name in data_centers}

# Start DC processing threads
for dc in dc_objects.values():
    threading.Thread(target=dc.process_requests, daemon=True).start()

# Start user threads
for region in user_regions:
    threading.Thread(target=user_simulator, args=(region, dc_objects)).start()


In [ ]:
import matplotlib.pyplot as plt

# Wait for all processing to finish
time.sleep(10)

# Collect data
names = []
requests_handled = []
avg_response = []

for dc in dc_objects.values():
    names.append(dc.name)
    requests_handled.append(dc.total_requests)
    avg = dc.total_response_time / dc.total_requests if dc.total_requests else 0
    avg_response.append(avg)

# Plot
plt.bar(names, avg_response)
plt.ylabel("Average Response Time (sec)")
plt.title("Multi-DC Response Time Comparison")
plt.show()
